<a href="https://colab.research.google.com/github/facebookresearch/habitat-sim/blob/main/examples/tutorials/colabs/ReplicaCAD_quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Habitat-sim ReplicaCAD Quickstart

This brief Colab tutorial demonstrates loading the ReplicaCAD dataset in Habitat-sim from a SceneDataset and rendering a short video of agent navigation with physics simulation.


In [1]:
import os
import sys

import magnum as mn

import habitat_sim
from habitat_sim.utils import viz_utils as vut

try:
    import ipywidgets as widgets
    from IPython.display import display as ipydisplay

    # For using jupyter/ipywidget IO components

    HAS_WIDGETS = True
except ImportError:
    HAS_WIDGETS = False

data_path = os.path.join(".", "data/")
output_path = os.path.join(".", "replica_cad_output/")
if not os.path.exists(output_path):
    os.mkdir(output_path)

# define some globals the first time we run.
if "sim" not in globals():
    global sim
    sim = None
    global obj_attr_mgr
    obj_attr_mgr = None
    global stage_attr_mgr
    stage_attr_mgr = None
    global rigid_obj_mgr
    rigid_obj_mgr = None

In [2]:
def make_cfg(settings):
    sim_cfg = habitat_sim.SimulatorConfiguration()
    sim_cfg.gpu_device_id = 0
    sim_cfg.scene_dataset_config_file = settings["scene_dataset"]
    sim_cfg.scene_id = settings["scene"]
    sim_cfg.enable_physics = settings["enable_physics"]
    # Specify the location of the scene dataset
    if "scene_dataset_config" in settings:
        sim_cfg.scene_dataset_config_file = settings["scene_dataset_config"]
    if "override_scene_light_defaults" in settings:
        sim_cfg.override_scene_light_defaults = settings[
            "override_scene_light_defaults"
        ]
    if "scene_light_setup" in settings:
        sim_cfg.scene_light_setup = settings["scene_light_setup"]

    # Note: all sensors must have the same resolution
    sensor_specs = []
    color_sensor_1st_person_spec = habitat_sim.CameraSensorSpec()
    color_sensor_1st_person_spec.uuid = "color_sensor_1st_person"
    color_sensor_1st_person_spec.sensor_type = habitat_sim.SensorType.COLOR
    color_sensor_1st_person_spec.resolution = [
        settings["height"],
        settings["width"],
    ]
    color_sensor_1st_person_spec.position = [0.0, settings["sensor_height"], 0.0]
    color_sensor_1st_person_spec.orientation = [
        settings["sensor_pitch"],
        0.0,
        0.0,
    ]
    color_sensor_1st_person_spec.sensor_subtype = habitat_sim.SensorSubType.PINHOLE
    sensor_specs.append(color_sensor_1st_person_spec)

    # Here you can specify the amount of displacement in a forward action and the turn angle
    agent_cfg = habitat_sim.agent.AgentConfiguration()
    agent_cfg.sensor_specifications = sensor_specs

    return habitat_sim.Configuration(sim_cfg, [agent_cfg])


def make_default_settings():
    settings = {
        "width": 1280,  # Spatial resolution of the observations
        "height": 720,
        "scene_dataset": "data/replica_cad/replicaCAD.scene_dataset_config.json",  # dataset path
        "scene": "NONE",  # Scene path
        "default_agent": 0,
        "sensor_height": 1.5,  # Height of sensors in meters
        "sensor_pitch": 0,  # sensor pitch (x rotation in rads)
        "seed": 1,
        "enable_physics": True,  # enable dynamics simulation
    }
    return settings


def make_simulator_from_settings(sim_settings):
    cfg = make_cfg(sim_settings)
    # clean-up the current simulator instance if it exists
    global sim
    global obj_attr_mgr
    global prim_attr_mgr
    global stage_attr_mgr
    global rigid_obj_mgr
    global metadata_mediator

    if sim != None:
        sim.close()
    # initialize the simulator
    sim = habitat_sim.Simulator(cfg)
    # Managers of various Attributes templates
    obj_attr_mgr = sim.get_object_template_manager()
    obj_attr_mgr.load_configs(str(os.path.join(data_path, "objects/example_objects")))
    prim_attr_mgr = sim.get_asset_template_manager()
    stage_attr_mgr = sim.get_stage_template_manager()
    # Manager providing access to rigid objects
    rigid_obj_mgr = sim.get_rigid_object_manager()
    # get metadata_mediator
    metadata_mediator = sim.metadata_mediator

    # UI-populated handles used in various cells.  Need to initialize to valid
    # value in case IPyWidgets are not available.
    # Holds the user's desired scene handle
    global selected_scene
    selected_scene = "NONE"


# [/setup]

In [3]:
# @title Define Simulation Utility Function { display-mode: "form" }
# @markdown (double click to show code)
def simulate(sim, dt=1.0, get_frames=True):
    # simulate dt seconds at 60Hz to the nearest fixed timestep
    print("Simulating {:.3f} world seconds.".format(dt))
    observations = []
    start_time = sim.get_world_time()
    while sim.get_world_time() < start_time + dt:
        sim.step_physics(1.0 / 60.0)
        if get_frames:
            observations.append(sim.get_sensor_observations())
    return observations

In [4]:
# @title Define Colab GUI Utility Functions { display-mode: "form" }
# @markdown (double click to show code)

# @markdown This cell provides utility functions to build and manage IPyWidget interactive components.

# Event handler for dropdowns displaying file-based object handles
def on_scene_ddl_change(ddl_values):
    global selected_scene
    selected_scene = ddl_values["new"]
    return selected_scene


# Build a dropdown list holding obj_handles and set its event handler
def set_handle_ddl_widget(scene_handles, sel_handle, on_change):
    sel_handle = scene_handles[0]
    descStr = "Available Scenes:"
    style = {"description_width": "300px"}
    obj_ddl = widgets.Dropdown(
        options=scene_handles,
        value=sel_handle,
        description=descStr,
        style=style,
        disabled=False,
        layout={"width": "max-content"},
    )

    obj_ddl.observe(on_change, names="value")
    return obj_ddl, sel_handle


def set_button_launcher(desc):
    button = widgets.Button(
        description=desc,
        layout={"width": "max-content"},
    )
    return button


# Builds widget-based UI components
def build_widget_ui(metadata_mediator):
    # Holds the user's desired scene
    global selected_scene
    selected_scene = "NONE"

    # Construct DDLs and assign event handlers
    # All file-based object template handles
    scene_handles = metadata_mediator.get_scene_handles()
    # If not using widgets, set as first available handle
    if not HAS_WIDGETS:
        selected_scene = scene_handles[0]
        return

    # Build widgets
    scene_obj_ddl, selected_scene = set_handle_ddl_widget(
        scene_handles,
        selected_scene,
        on_scene_ddl_change,
    )

    # Display DDLs
    ipydisplay(scene_obj_ddl)


if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument("--no-display", dest="display", action="store_false")
    parser.add_argument("--no-make-video", dest="make_video", action="store_false")
    parser.set_defaults(show_video=True, make_video=True)
    args, _ = parser.parse_known_args()
    show_video = args.display
    display = args.display
    make_video = args.make_video
else:
    show_video = False
    make_video = False
    display = False

# View ReplicaCAD in Habitat-sim
Use the code in this section to view assets in the Habitat-sim engine.

In [5]:
# [initialize]
# @title Initialize Simulator{ display-mode: "form" }

sim_settings = make_default_settings()
make_simulator_from_settings(sim_settings)
# [/initialize]

I0114 13:39:54.086328 15156608 ManagedFileBasedContainer.h:210] <Dataset>::convertFilenameToPassedExt : Filename : default changed to proposed scene_dataset_config.json filename : default.scene_dataset_config.json
I0114 13:39:54.086417 15156608 AttributesManagerBase.h:365] <Dataset>::createFromJsonOrDefaultInternal : Proposing JSON name : default.scene_dataset_config.json from original name : default | This file  does not exist.
I0114 13:39:54.086504 15156608 AssetAttributesManager.cpp:120] Asset attributes (capsule3DSolid : capsule3DSolid_hemiRings_4_cylRings_1_segments_12_halfLen_0.75_useTexCoords_false_useTangents_false) created and registered.
I0114 13:39:54.086550 15156608 AssetAttributesManager.cpp:120] Asset attributes (capsule3DWireframe : capsule3DWireframe_hemiRings_8_cylRings_1_segments_16_halfLen_1) created and registered.
I0114 13:39:54.086591 15156608 AssetAttributesManager.cpp:120] Asset attributes (coneSolid : coneSolid_segments_12_halfLen_1.25_rings_1_useTexCoords_fals

Renderer: Apple M1 by Apple
OpenGL version: 4.1 Metal - 76.1
Using optional features:
    GL_ARB_vertex_array_object
    GL_ARB_ES2_compatibility
    GL_ARB_separate_shader_objects
    GL_ARB_texture_storage
    GL_EXT_texture_filter_anisotropic
    GL_EXT_debug_label
    GL_EXT_debug_marker
Using driver workarounds:
    no-layout-qualifiers-on-old-glsl
    apple-buffer-texture-unbind-on-buffer-modify


 attributes created and registered.
I0114 13:39:54.088037 15156608 MetadataMediator.cpp:127] ::createSceneDataset : Dataset data/replica_cad/replicaCAD.scene_dataset_config.json successfully created.
I0114 13:39:54.088043 15156608 MetadataMediator.cpp:230] ::setActiveSceneDatasetName : Attempt to create new dataset data/replica_cad/replicaCAD.scene_dataset_config.json  succeeded. Currently active dataset : data/replica_cad/replicaCAD.scene_dataset_config.json
I0114 13:39:54.088047 15156608 MetadataMediator.cpp:183] ::setCurrPhysicsAttributesHandle : Old physics manager attributes  changed to ./data/default.physics_config.json successfully.
I0114 13:39:54.088054 15156608 MetadataMediator.cpp:68] ::setSimulatorConfiguration : Set new simulator config for scene/stage : NONE and dataset : data/replica_cad/replicaCAD.scene_dataset_config.json which is currently active dataset.
I0114 13:39:54.219983 15156608 ManagedFileBasedContainer.h:210] <Scene Instance>::convertFilenameToPassedExt : File

In [6]:
# @title Select a SceneInstance: { display-mode: "form" }
# @markdown Select a scene from the dropdown and then run the next cell to load and simulate that scene and produce a visualization of the result.

build_widget_ui(sim.metadata_mediator)

## Load the Select Scene and Simulate!
This cell will load the scene selected above, simulate, and produce a visualization.

In [7]:
global selected_scene
if sim_settings["scene"] != selected_scene:
    sim_settings["scene"] = selected_scene
    make_simulator_from_settings(sim_settings)

observations = []
start_time = sim.get_world_time()
while sim.get_world_time() < start_time + 4.0:
    sim.agents[0].scene_node.rotate(mn.Rad(mn.math.pi_half / 60.0), mn.Vector3(0, 1, 0))
    sim.step_physics(1.0 / 60.0)
    if make_video:
        observations.append(sim.get_sensor_observations())

# video rendering of carousel view
video_prefix = "ReplicaCAD_scene_view"
if make_video:
    vut.make_video(
        observations,
        "color_sensor_1st_person",
        "color",
        output_path + video_prefix,
        open_vid=show_video,
        video_dims=[1280, 720],
    )

Encoding the video: ./replica_cad_output/ReplicaCAD_scene_view.mp4 
